"""
# Electric Vehicle Market & Consumer Insights Analysis

# **Table of Contents**

## **1️⃣ Market & Consumer Insights**
1. [Most popular EV models and manufacturers](#q1)  
2. [EV adoption by county, city, and postal code](#q2)  
3. [Popularity trends by model year](#q3)  
4. [Electric range variation by manufacturer and model](#q4)  
5. [BEV vs. PHEV percentage distribution](#q5)  
6. [EV ownership distribution based on census tract data](#q6)  

## **2️⃣ Infrastructure & Utility Planning**
7. [Which areas have the highest number of EVs, indicating a **need for more charging stations**?](#q7)  
8. [Correlation of EV distribution with electric utilities](#q8)  
9. [Electric utilities serving the most EV users](#q9)  
10. [Electric range variation across regions](#q10)  
11. [Counties with the highest concentration of long-range EVs](#q11)  
12. [Regions preferring PHEVs over BEVs](#q12)  

## **3️⃣ Financial & Policy Impact Analysis**
13. [Base MSRP variation by vehicle type and model](#q13)  
14. [Price distribution of CAFV-eligible vs. non-eligible vehicles](#q14)  
15. [Higher-priced EVs in specific counties or census tracts](#q15)  
16. [CAFV eligibility impact on EV sales trends](#q16)  
17. [Counties with higher adoption of CAFV-eligible vehicles](#q17)  
18. [EV market share changes over time](#q18)  
19. [Manufacturers/models with highest CAFV eligibility](#q19)  

## **4️⃣ Environmental & Regulatory Considerations**
20. [Impact of policies/incentives on EV adoption rates](#q20)  

---

## Column Descriptions

- **VIN (1-10)**: The first 10 characters of the Vehicle Identification Number (VIN), uniquely identifying each vehicle.
- **County**: The county where the vehicle is registered, useful for analyzing EV adoption at a county level.
- **City**: The city where the vehicle is registered, providing insights at a local level.
- **State**: The state where the vehicle is registered (e.g., "WA" for Washington).
- **Postal Code**: The ZIP code of the registered vehicle, aiding demographic and geographic analysis.
- **Model Year**: The manufacturing year of the vehicle, helping track the age of EVs on the road.
- **Make**: The vehicle manufacturer (e.g., Tesla, Chevrolet, BMW).
- **Model**: The specific model of the vehicle (e.g., Model S, Bolt EV, i3).
- **Electric Vehicle Type**:
  - **Battery Electric Vehicle (BEV)**: Fully electric, no gasoline engine.
  - **Plug-in Hybrid Electric Vehicle (PHEV)**: Contains both a battery-powered motor and a gasoline engine.
- **Clean Alternative Fuel Vehicle (CAFV) Eligibility**: Indicates if the vehicle qualifies as a Clean Alternative Fuel Vehicle based on government regulations.
- **Electric Range**: The estimated number of miles the vehicle can travel on a full charge.
- **Base MSRP**: The Manufacturer’s Suggested Retail Price (MSRP) of the vehicle, useful for price trend analysis.
- **Legislative District**: The district where the vehicle is registered, useful for policy and regulation tracking.
- **DOL Vehicle ID**: Unique identifier assigned to the vehicle by the Department of Licensing (DOL).
- **Vehicle Location**: Geographic coordinates (latitude and longitude) indicating where the vehicle is registered.
- **Electric Utility**: The name of the electricity provider for the registered location, useful for analyzing EV charging infrastructure.
- **2020 Census Tract**: A geographic identifier used in census data, aiding demographic analysis related to EV ownership.

---
"""


### Importing the libraries

In [ ]:
#Imporing the liunraries
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go

import scipy.stats as stats


import warnings


## **Loading The Dataset**

In [ ]:
df = pd.read_csv('Electric_Vehicle_Population_Data.csv')  #loading the Data 
df.head() #Displaying the first 5 rows of data


## Data Preprocessing

In this section, we perform data cleaning and preprocessing to prepare the dataset for analysis.  

In [ ]:
df.shape #Sape of data

In [ ]:
# information about the data
df.info()

### Processing the null Values

In [ ]:
#checking the null values presentage in data
percentage_missingVal = (df.isna().sum() / len(df) * 100).round(2).to_frame(name='null Percentage' )
percentage_missingVal.index.name = 'Column Name'
percentage_missingVal

In [ ]:
#Comparing the both colums 
df['Legislative District'].equals(df['Legislative District']) 

"""
The above code will return True if the two columns are equal, and False if not.
so Base on Its results we can say that both columns represents samp infornmation so we can deleat one of them
"""
df.drop('Legislative District', axis=1 , inplace=True) # deleating the column


In [ ]:
# column in lst are the columns less than 1 percentage of missinf valaues
imput = SimpleImputer(strategy='mean') # replacng null values with mean of that column
lst = ['Base MSRP', 'Electric Range']

for col in lst:
    df[col] = imput.fit_transform(df[[col]])


# Other columns have  very leas amoount of missing values so we can dero the , it not going to impact the data
df.dropna(inplace=True)


"""
    Now We Do not have any Missing values in data
"""

In [ ]:
#checking null values in data
df.isna().sum()

In [ ]:
print(df['Clean Alternative Fuel Vehicle (CAFV) Eligibility'].unique())

# Replacing the values in the column to make it more Readable
df.loc[:, 'Clean Alternative Fuel Vehicle (CAFV) Eligibility'] = df['Clean Alternative Fuel Vehicle (CAFV) Eligibility'].replace({
    'Not eligible due to low battery range': 'Not eligible',
    'Clean Alternative Fuel Vehicle Eligible': 'Eligible',
    'Eligibility unknown as battery range has not been researched': 'Unknown'
})


In [ ]:
# Splitting the  Vehicle Location column to get the Latitude and Longitude
df['Longitude'] = df['Vehicle Location'].apply(lambda x : x.split(' ' )[1][1:] if pd.notnull(x) else None)
df['Latitude'] = df['Vehicle Location'].apply(lambda x : x.split(' ')[2][:-1] if pd.notnull(x) else None)

# Droping the Vehicle Location column
df.drop('Vehicle Location', axis=1, inplace=True)

## **Exploratory Data Analysis (EDA)**

Exploratory Data Analysis (EDA) helps in understanding the dataset through summary statistics and visualizations.  


### **1️⃣ Market & Consumer Insights**

#### 1. What are the most popular **electric vehicle (EV) models** and manufacturers in different counties/cities?

In [ ]:
# Group by County, Make, and Model to get the count of EVs
top_ev_city = df.groupby(['City', 'Make', 'Model']).size().reset_index(name='Ev Count')
top_ev_city['Car Model'] = top_ev_city['Make'] + ' ' + top_ev_city['Model']
top_ev_city.drop(['Make', 'Model'], axis=1, inplace=True)

# Group by County and sum the EV counts
top_ev_city = top_ev_city.groupby(['City'])['Ev Count'].sum().reset_index().sort_values(by='Ev Count', ascending=False).head(10)

# Visualization
fig = px.bar(top_ev_city, x='City', y='Ev Count', text='Ev Count', title='Top 10 Cities with Highest Number of EVs',
             labels={'County': 'County', 'Ev Count': 'Number of EVs'})
fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)', marker_line_width=1.5, textposition='outside')
fig.update_layout(plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='white', font=dict(size=14), xaxis=dict(tickangle=-45), yaxis=dict(showgrid=True, gridcolor='lightgray'))
fig.show()


 ### **1. Seattle Leads EV Adoption**  
- Seattle has **35,864 EVs**, significantly higher than other cities.  
- **Action:** Expand **charging networks** and **EV incentives** to sustain growth.  

### **2. Growth Potential in Mid-Tier Cities**  
- Bellevue (10,966), Vancouver (8,103), and Redmond (7,772) show **moderate adoption**.  
- **Action:** Invest in **charging infrastructure, corporate EV programs, and awareness campaigns**.  

### **3. Low-Adoption Cities Need Strategic Focus**  
- Olympia (5,419) and Tacoma (4,818) have **lower EV adoption** due to potential barriers.  
- **Action:** Implement **localized incentives, subsidized home chargers, and public EV policies**.  

### **4. Infrastructure & Long-Term Strategy**  
- **Seattle’s high adoption** requires **fast-charging expansion** and smart-grid integration.  
- **Action:** Use **data-driven demand analysis** to plan charging station growth.  

### **Key Recommendations**  
✅ Expand **EV charging networks** in high-adoption cities.  
✅ Launch **incentives & promotional campaigns** in mid-tier cities.  
✅ Address adoption barriers in **low-EV cities** through research and targeted policies.  

These actions will drive **sustainable EV adoption and long-term market growth**.  


In [ ]:
# Group by County and Make, then count EV registrations
county_make_counts = df.groupby(["County", "Make"]).size().reset_index(name="Ev Count")

# Find the manufacturer with the highest count per county
top_manufacturer_per_county = county_make_counts.loc[county_make_counts.groupby("County")["Ev Count"].idxmax()]

# Sort by highest count for better readability
top_manufacturer_per_county = top_manufacturer_per_county.sort_values(by="Ev Count", ascending=False)

# Display the top manufacturers in each county
top_manufacturer_per_county.head(10).style.background_gradient(cmap='Reds')


In [ ]:
# Group by city and Make, then count EV registrations
top_manufaturer_city = df.groupby(['City','Make']).size().reset_index(name='Ev Count')
# manufacturer with highest count
top_manufaturer_city = top_manufaturer_city.loc[top_manufaturer_city.groupby('City')['Ev Count'].idxmax()]
#sortin
top_manufaturer_city = top_manufaturer_city.sort_values(by='Ev Count' , ascending=False).head(10)
# Display the top manufacturers in each city
top_manufaturer_city.style.background_gradient(cmap='Purples')


### **Key Observations:**  
1. **Seattle Leads Tesla Registrations**  
   - With **13,191 Tesla EVs**, Seattle has the **highest adoption**, indicating strong brand presence and charging infrastructure.  

2. **Tesla Dominates Across Multiple Cities**  
   - Bellevue (**6,418**), Redmond (**4,497**), and Bothell (**4,223**) follow, showing Tesla's strong market share.  
   - This suggests Tesla’s **widespread popularity** in urban and suburban areas.  

3. **Growth Opportunities in Mid-Sized Cities**  
   - Cities like Vancouver (**3,179**) and Renton (**3,048**) show **moderate adoption**, indicating potential for further growth with incentives and infrastructure expansion.  

4. **Strategic Expansion for Tesla**  
   - In lower-ranked cities like **Kent (2,070) and Lynnwood (2,014)**, increasing **charging infrastructure, dealership presence, and localized incentives** can boost adoption.  

### **Recommendations:**  
✅ **Enhance Tesla Supercharger networks** in high-demand cities.  
✅ **Target marketing & financing options** for mid-tier cities.  
✅ **Expand presence in emerging EV hubs** (Kent, Lynnwood).  

Tesla's dominance highlights the **importance of infrastructure, consumer trust, and premium EV offerings** in driving adoption.  


#### 2. How does the **adoption of EVs vary by county, city, and postal code**?

In [ ]:
# Grouping and sorting EV adoption by County, City, and Postal Code
city_ev_adoption = df.groupby(['County', 'City', 'Postal Code']).size().reset_index(name='EV Count')
city_ev_adoption = city_ev_adoption.sort_values(by='EV Count', ascending=False)

city_ev_adoption['Postal Code'] = city_ev_adoption['Postal Code'].astype(int)
# Apply a sequential color gradient only to 'EV Count'
styled_df = city_ev_adoption.head(10).style.background_gradient(subset=['EV Count'], cmap='YlOrRd')

styled_df

### **EV Adoption Insights by County, City, and Postal Code**  

### **1. King County Dominates EV Adoption**  
- Majority of high EV adoption areas are in **King County**, with cities like **Redmond (5,499), Kirkland (3,590), and Bellevue (3,463)** leading.  
- Suggests **better infrastructure, higher income demographics, and strong EV policies**.  

### **2. Snohomish County Shows Growth Potential**  
- **Bothell (3,309) in Snohomish County** ranks high, indicating increasing EV adoption beyond King County.  
- Potential for **charging infrastructure expansion and incentive programs**.  

### **3. Postal Code Variations Indicate Localized Trends**  
- Certain ZIP codes (98052, 98033, 98006) have higher EV counts, implying **localized policy effectiveness and consumer preference shifts**.  
- High-income regions may have **greater early adoption**, requiring **scaling strategies** for broader accessibility.  

### **Recommendations**  
✅ Expand **charging stations** in emerging EV hubs.  
✅ Offer **targeted incentives** in lower-adoption regions.  
✅ Use **ZIP code-level insights** for localized marketing and infrastructure planning.  

This data can help optimize **EV expansion strategies and regional policy-making**. 🚗⚡  


### 3. Which vehicle makes and models are growing in popularity over different **model years**?

In [ ]:
# Gropping Model year , Make , model to get the popularity of each make and its model over the years
Model_popularity_byYear = df.groupby(['Model Year','Make', 'Model']).size().reset_index(name='EVs').sort_values(by='Model Year' )
#Creating a new column which includes Manufacturer and Thier Evs model 
Model_popularity_byYear['Comapny Model'] = Model_popularity_byYear['Make'] + ' ' + Model_popularity_byYear['Model']
# Now we have company Model column so we can deop mak and model column
Model_popularity_byYear.drop(['Make','Model'] , axis=1, inplace=True)
Model_popularity_byYear

In [ ]:
# Create a line chart
fig = px.line(
    Model_popularity_byYear, 
    x="Model Year", 
    y="EVs", 
    color="Comapny Model",
    title="EV Model Popularity Growth Over the Years",
    labels={"Model Year": "Year", "EVs": "Number of EVs", "Comapny Model": "Manufacturer & Model"},
    markers=True
)

# Update layout for better readability & larger size
fig.update_layout(
    xaxis_title="Model Year",
    yaxis_title="Number of EVs",
    legend_title="EV Models",
    hovermode="x unified",
    template="plotly_white",
    width=1200,  # Increase width
    height=700   # Increase height
)

# Show the plot
fig.show()


### **EV Make & Model Popularity Trends Over Model Years**  

### **1. Early Adoption (1999 - 2010)**  
- **Ford Ranger (1999-2000)** and **Toyota RAV4 (2002-2003)** were among the **earliest EV models**, indicating limited options and lower adoption.  
- **Tesla Roadster (2008)** marked the start of premium EVs gaining traction.  

### **2. Growth Phase (2010 - 2020)**  
- Major EV adoption surge seen post-2010 with **Tesla, Nissan, and Chevrolet** leading the market.  
- Widening model range shows increasing consumer interest and manufacturer investment.  

### **3. Recent & Future Growth (2020 - 2025)**  
- **Kia (EV9, Niro, Sorento) and Genesis (GV60)** emerging as strong contenders.  
- **Luxury brands like Volvo (XC90) are expanding EV offerings**, signaling market diversification.  

### **Key Takeaways**  
✅ **Tesla pioneered** modern EV adoption; legacy brands are now catching up.  
✅ **SUVs & crossovers (e.g., Kia Sorento, Volvo XC90)** are gaining popularity.  
✅ **Future growth expected across multiple brands**, reflecting **increased competition & EV affordability**.  

Understanding these trends helps manufacturers, policymakers, and dealers **align strategies for future EV demand**. 🚗⚡  


### 4. How does the **electric range** vary among different manufacturers and models?

In [ ]:
# gropping electric range , model and manufacturer to get their variation in range
variation_in_electric_range = df[['Electric Range','Model','Make']].sort_values(by='Electric Range' , ascending=False)
#dropping dublicate to get only distincts
variation_in_electric_range.drop_duplicates(subset='Model')

In [ ]:
# Visulistion For Elctric range of  EV model and their manufacturer
fig = px.scatter(
    variation_in_electric_range,
    x="Make",
    y="Electric Range",
    color="Make",
    size="Electric Range",
    hover_data=["Model"],
    title="Electric Range Distribution by Manufacturer",
    labels={"Electric Range": "Electric Range (miles)", "Make": "Manufacturer"},
)

fig.update_layout(template="plotly_white", width=1000, height=600)
fig.show()


### **Electric Range Variation Across Manufacturers & Models**  

### **1. High-Range EVs Dominate the Market**  
- **Top manufacturers (Tesla, Lucid, Rivian)** offer the **longest electric ranges**, exceeding **300-500 miles per charge**.  
- These models cater to **long-distance travelers and premium EV buyers**.  

### **2. Mid-Range EVs Gaining Popularity**  
- **Brands like Hyundai, Kia, and Ford** offer EVs with **200-300 miles range**, balancing affordability with range efficiency.  
- Ideal for **urban and suburban drivers** who need reliable daily commuting options.  

### **3. Lower-Range EVs & City Cars**  
- **Early EV models and budget-friendly brands** still provide **<150 miles of range**.  
- Best suited for **short-distance city commuting and fleet operations**.  

### **Key Takeaways**  
✅ **Long-range EVs** (Tesla, Lucid) lead, catering to premium markets.  
✅ **Mid-range EVs** (Kia, Hyundai, Ford) balance cost and range, expanding accessibility.  
✅ **Short-range EVs** still relevant for **urban use and affordability**.  

Understanding **range variations helps buyers and policymakers** optimize **charging infrastructure and market positioning**. 🚗⚡  


### 5. What percentage of vehicles are **Battery Electric Vehicles (BEV)** vs. **Plug-in Hybrid Electric Vehicles (PHEV)**?

In [ ]:
# Count each type of Electric Vehicle
evs_battery_type = df['Electric Vehicle Type'].value_counts().reset_index()
evs_battery_type.columns = ['Electric Vehicle Type', 'Count']

# Create the pie chart
fig = px.pie(
    evs_battery_type,
    values="Count",
    names="Electric Vehicle Type",
    title="Electric Vehicle Distribution by Battery Type",
    hole=0.4,  # Creates a donut chart for better visualization
    color_discrete_sequence=px.colors.sequential.Blues_r  # Better color scheme
)

# Improve layout for better readability
fig.update_traces(textinfo="percent+label", pull=[0.05] * len(evs_battery_type))  # Slightly pull out slices for emphasis
fig.update_layout(width=800, height=500, showlegend=True)

# Show the plot
fig.show()

### **Electric Vehicle Distribution by Battery Type**  

### **1. BEVs Dominate the Market**  
- **Battery Electric Vehicles (BEVs) make up 79.1%** of the total EV market.  
- BEVs operate **entirely on electricity**, reducing carbon emissions and **eliminating reliance on gasoline**.  

### **2. PHEVs Hold a Smaller Share**  
- **Plug-in Hybrid Electric Vehicles (PHEVs) account for 20.9%** of the total EV fleet.  
- These vehicles use both **electric power and gasoline**, offering flexibility but still contributing to fuel consumption.  

### **Key Takeaways**  
✅ **Majority adoption of BEVs** signals growing confidence in all-electric mobility.  
✅ **PHEVs remain relevant** for consumers needing extended range with gas backup.  
✅ **Policy & infrastructure focus should prioritize charging networks** to support full BEV transition.  

This insight helps guide **future investments in charging infrastructure, incentives, and EV technology development**. 🚗⚡  


### 6. What is the distribution of **EV ownership based on census tract data**?

In [ ]:
# Grouping data by 2020 Census Tract and counting EVs in each tract
census_distribution = df["2020 Census Tract"].value_counts().sort_values(ascending=False)

# Plotting the distribution
plt.figure(figsize=(12, 6))
plt.hist(census_distribution, bins=50, color='blue', edgecolor='black', alpha=0.7)
plt.xlabel("Number of EVs per Census Tract")
plt.ylabel("Frequency (Number of Census Tracts)")
plt.title("Distribution of EV Ownership Across Census Tracts")
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()


### **Distribution of EV Ownership Across Census Tracts**  

### **1. Skewed Distribution of EV Ownership**  
- The histogram shows that **most census tracts have low EV ownership**, with **a few tracts having significantly higher numbers**.  
- A **right-skewed distribution** indicates that EV adoption is concentrated in specific areas rather than evenly spread.  

### **2. Majority of Census Tracts Have Low EV Counts**  
- Most census tracts have **fewer than 100 EVs**.  
- A few tracts, however, have **over 1000 EVs**, likely reflecting **urban centers with better charging infrastructure and higher income levels**.  

### **Key Takeaways**  
✅ **EV adoption is not uniform**—some areas have significantly higher penetration than others.  
✅ **Targeted policies** and **incentives** can help encourage adoption in lower-adoption areas.  
✅ **Expanding charging infrastructure** in underserved census tracts could boost adoption rates.  

These insights can **help guide infrastructure development, incentives, and EV promotion strategies** to ensure equitable adoption. 🚗⚡  


### **2️⃣ Infrastructure & Utility Planning**

### 7. Which areas have the highest number of EVs, indicating a **need for more charging stations**?

In [ ]:
# Getting count of evs for each Postal Code indecating the need of charging station
area_ev_count = df.groupby(['City' , 'Postal Code']).size().reset_index(name='Evs Count').sort_values(by='Evs Count' , ascending=False)
#typecasting the postal code colum
area_ev_count['Postal Code'] = area_ev_count['Postal Code'].astype(int)
#Displaying only top 20 cities whith their postal code
area_ev_count.head(20).style.background_gradient(subset='Evs Count',cmap='Reds')

### **High EV Adoption Areas & Charging Station Demand**  

### **1. Cities & Postal Codes with the Highest EV Counts**  
- **Redmond (98052) leads with 5,499 EVs**, indicating a **strong demand for more charging stations**.  
- Other high-adoption areas include **Kirkland (98033), Bellevue (98006), Bothell (98012), and Seattle (98115)**, all with **3,000+ EVs**.  

### **2. Implications for Charging Infrastructure**  
🚗 **Urban & suburban areas need expanded charging networks** to meet growing demand.  
⚡ **Redmond, Bellevue, Kirkland, and Seattle require priority investment** in fast-charging stations.  
🏡 **Residential & workplace charging solutions** can enhance convenience in these high-adoption areas.  

### **Key Takeaways**  
✅ **Infrastructure planning should focus on high-EV-count areas** for better accessibility.  
✅ **Encouraging public-private partnerships** can accelerate charging station expansion.  
✅ **Data-driven expansion** ensures optimal placement of new charging stations.  

📌 **Next Steps:** Identify specific charging gaps & propose strategic locations for new installations! 🚀⚡  


### 8. How is the distribution of EVs correlated with **different electric utilities**?

In [ ]:
# Preprocessing the "Electric Utility" column
# Function to clean and standardize the electric utility names
def clean_utility(utility):
    if pd.isna(utility):
        return "Unknown"
    utility = utility.title()  # Convert to title case for consistency
    utility = utility.replace("||", ", ").replace("|", ", ")  # Standardize separators
    return utility.strip()

# Apply cleaning function
df["Electric Utility"] = df["Electric Utility"].apply(clean_utility)

# Group by cleaned electric utility names and count EVs
ev_by_cleaned_utility = df.groupby("Electric Utility").size().reset_index(name="Evs Count").sort_values(by="Evs Count", ascending=False)


# Create scatter plot
fig = px.scatter(
    ev_by_cleaned_utility,
    x="Evs Count",
    y="Electric Utility",
    size="Evs Count",
    title="EV Distribution Across Electric Utilities",
    labels={"Evs Count": "Number of EVs", "Electric Utility": "Electric Utility Provider"},
    template="plotly_dark",
    color='Evs Count'
)

# Show the plot
fig.show()



In [ ]:
# Create a contingency table (EV counts per Electric Utility)
contingency_table = ev_by_cleaned_utility["Evs Count"].values

# Perform the Chi-Square Test
chi2_stat, p_value = stats.chisquare(contingency_table)

# Print results
print(f"Chi-Square Statistic: {chi2_stat:.2f}")
print(f"P-Value: {p_value:.5f}")

# Interpretation
if p_value < 0.05:
    print("✅ The distribution of EVs across different electric utilities is statistically significant.")
else:
    print("❌ The distribution of EVs across different electric utilities is NOT significantly different.")


### 🔌 EV Distribution Across Electric Utilities

#### 🔍 Key Observations  
- **Bonneville Power Administration** serves multiple regions with high EV adoption.  
- **Puget Sound Energy Inc. & City of Tacoma (WA)** have the highest EV counts.  
- **Smaller utilities** have lower EV penetration.  

## ⚡ Implications  
- **Expand charging infrastructure** in high-demand areas.  
- **Optimize grid load** with demand-based pricing.  
- **Promote adoption** in low-EV regions via incentives.  



### 9. Are certain **electric utilities serving more EV users** than others?

### 🔌 EV Distribution Among Electric Utilities

Some electric utilities serve a significantly higher number of EV users compared to others. This could be due to factors such as:
- **Urban vs. Rural Distribution:** Higher EV adoption in urban areas with better charging infrastructure.
- **Utility Policies & Incentives:** Some utilities offer better incentives for EV adoption.
- **Grid Capacity & Coverage:** Larger utilities with extensive coverage naturally serve more EVs.

📊 **Insight:** Identifying high-density EV areas can help utilities optimize grid management and charging infrastructure planning.


### 10. How does the **electric range of EVs** vary across different regions?

In [ ]:
# Grouping data by County and State to get the sum of Electric Range
electric_range_by_region = df.groupby(['County', 'State'])['Electric Range'].mean().reset_index()

# Sorting the data by Electric Range in descending order
electric_range_by_region = electric_range_by_region.sort_values(by='Electric Range',
                                                                 ascending=False).head(20)

# Visualization
fig = px.bar(
    electric_range_by_region,
    x='County',
    y='Electric Range',
    color='State',
    title='Top 20 Counties by Total Electric Range of EVs',
    labels={'County': 'County', 'Electric Range': 'Total Electric Range (miles)'},
    text='Electric Range'
)

fig.update_traces(textposition='outside')
fig.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='white',
    font=dict(size=14),
    xaxis=dict(tickangle=-45),
    yaxis=dict(showgrid=True, gridcolor='lightgray')
)

fig.show()

The visualization above highlights the **top 20 counties** with the highest total electric range of EVs. **St. Louis, MO** leads with the highest total electric range of **322 miles**, followed by **Rockingham, NH (308 miles)** and **Palm Beach, FL (291 miles)**. 

This indicates that certain regions have a higher concentration of long-range EVs, possibly due to **better charging infrastructure, incentives, or consumer preferences for higher-range electric vehicles**.


### 11. Which counties have the **highest concentration of long-range EVs**, suggesting early adoption trends?

In [ ]:
long_range_evs_county = df.groupby(['County'])['Electric Range'].mean().reset_index(name='Range').sort_values(by='Range', ascending=False)
long_range_evs_county.head(10).style.background_gradient(cmap='Blues')


Counties like **St. Louis (322 miles), Rockingham (308 miles), and Palm Beach (291 miles)** have the highest average EV range, indicating early adoption trends. These areas likely have **better charging infrastructure, incentives, or a preference for long-range EVs**, driving higher adoption rates.


### 12. Are some regions **more inclined to adopt PHEVs rather than BEVs**, indicating a lack of charging infrastructure?

In [ ]:
# Countung the each electric type accurance
battery_infra = pd.DataFrame(df['Electric Vehicle Type'].value_counts())
#renaming the column
battery_infra.columns = ['Evs Count']
#Sum of all the evs count
tot = np.sum(battery_infra['Evs Count'])
#percentage of each electric vehicle type
battery_infra['Perc'] = (battery_infra['Evs Count']*100 / tot)
# displaying
battery_infra.style.background_gradient(cmap='Reds')


- **BEVs constitute 79.1%** of the total EVs, while **PHEVs make up 20.9%**.  
- A **higher proportion of PHEVs in certain regions** suggests concerns about **charging infrastructure availability**.  
- PHEVs provide flexibility by using both electricity and fuel, making them preferable where **charging access is limited**.  
- **Expanding public charging networks** could encourage a shift from PHEVs to BEVs, accelerating EV adoption.  

In [ ]:
# No. Of BEV Vehicles in each county
BEV_ev= df[df['Electric Vehicle Type'] == 'Battery Electric Vehicle (BEV)']

# counting the each accurance of county
ele_type = BEV_ev['County'].value_counts()
ele_type = pd.DataFrame(ele_type)

# Find counties where EV count is above average
average_ev_count = ele_type['count'].mean()
above_average_counties = ele_type[ele_type['count'] > average_ev_count]

above_average_counties


- **High PHEV adoption in certain counties** suggests reliance on plug-in hybrids due to **insufficient charging infrastructure**.  
- **Counties with above-average BEV adoption** indicate a **well-developed charging network** and greater acceptance of fully electric vehicles.  
- Comparing PHEV-dominant counties with BEV-rich areas can **pinpoint locations needing charging infrastructure expansion**.  
- **Targeted investment in public charging stations** could shift PHEV users toward BEVs, driving full electrification.  

In [ ]:
#Getting the count
porr_battery_infra=df[df['Electric Vehicle Type'] == 'Plug-in Hybrid Electric Vehicle (PHEV)']
porr_battery_infra=porr_battery_infra['County'].value_counts().to_frame()
porr_battery_infra


- **King County leads in PHEV adoption** with **21,144 vehicles**, highlighting a potential **need for better charging facilities**.  
- **Snohomish (4,497), Pierce (3,957), Clark (3,372), and Thurston (1,980) Counties** also show significant PHEV numbers.  
- A **higher PHEV concentration** suggests that **drivers may be hesitant to switch to BEVs** due to limited charging availability.  
- **Investing in fast-charging stations** in these regions can encourage a shift toward fully electric vehicles.  


In [ ]:
Poor_last = porr_battery_infra.index.unique() #countiest with PHEV evs
lst = list(above_average_counties.index)  # Above Avg BEV adoption rate counties

In [ ]:
# Countries that are above avg BEH but still good no. of peoples are cosnsidering battery as along with gas engine
for county in lst:
    if county in Poor_last:
            print(county) #


- **Key Counties:** King, Snohomish, Pierce, Clark, Thurston, Kitsap, Spokane, Whatcom, Benton, Skagit, Island, Chelan, Yakima.  
- Despite having an **above-average number of BEVs**, these counties still show a **considerable number of PHEV users**.  
- This suggests that **residents may perceive charging infrastructure as insufficient**, leading them to prefer PHEVs over BEVs.  
- **Expanding fast-charging networks** in these areas could accelerate BEV adoption and reduce reliance on gasoline.  

### **3️⃣ Financial & Policy Impact Analysis**

### 13. How does the **Base MSRP (Manufacturer Suggested Retail Price)** vary by vehicle type and model?

In [ ]:
suggested_base_msrp=df.groupby(['Make','Model'])['Base MSRP'].mean().reset_index(name="suggested Base_MSRP").sort_values(by='suggested Base_MSRP', 
                                                                                                                         ascending=False).round()
# Remove zero MSRP values
dsuggested_base_msrp_filtered = suggested_base_msrp[suggested_base_msrp["suggested Base_MSRP"] > 0].head(10)
# Plotting the bar chart using plotly
fig = px.bar(
    dsuggested_base_msrp_filtered,
    x=dsuggested_base_msrp_filtered['Make']  +  ' '  + dsuggested_base_msrp_filtered['Model'],
    y='suggested Base_MSRP',
    title='Suggested Base MSRP of Electric Vehicles by Make and Model',
    labels={'Make': 'Make', 'suggested Base_MSRP': 'Suggested Base MSRP'},
    text='suggested Base_MSRP'
)

fig.update_layout(
    xaxis_title='Make',
    yaxis_title='Suggested Base MSRP',
    legend_title='Model',
    template='plotly_white',
    xaxis=dict(tickangle=-45)
)

fig.show()

- **Porsche 918** stands out with a significantly high MSRP of **$845,000**, making it a luxury EV.  
- **Tesla Roadster** and **Fisker Karma** follow at **$103,178** and **$102,000**, indicating premium pricing.  
- Most mainstream EVs, such as the **Kia Soul EV ($32,363)** and **Subaru Crosstrek ($25,211)**, have significantly lower price points.  
- The wide pricing gap suggests a **diverse EV market**, catering to both high-end luxury and affordable consumer segments.  
- **Luxury EVs dominate the high price range**, while **mass adoption is driven by lower-cost models**.  

### 14. What is the price distribution of **clean alternative fuel eligible vs. non-eligible vehicles**?

In [ ]:
# Filter the dataframe to include only relevant columns
price_distribution = df[['Base MSRP', 'Clean Alternative Fuel Vehicle (CAFV) Eligibility']]

# Create a box plot to visualize the price distribution
fig = px.box(
    price_distribution,
    x='Clean Alternative Fuel Vehicle (CAFV) Eligibility',
    y='Base MSRP',
    title='Price Distribution of Clean Alternative Fuel Eligible vs. Non-Eligible Vehicles',
    labels={'Clean Alternative Fuel Vehicle (CAFV) Eligibility': 'CAFV Eligibility', 'Base MSRP': 'Base MSRP'},
    color='Clean Alternative Fuel Vehicle (CAFV) Eligibility'
)

# Update layout for better readability
fig.update_layout(
    xaxis_title='CAFV Eligibility',
    yaxis_title='Base MSRP',
    template='plotly_white',
    width=800,
    height=500
)

# Show the plot
fig.show()

In [ ]:
price_distribution[price_distribution['Base MSRP'] > 0] 

- **Most eligible vehicles** have their **MSRP below $100,000**, suggesting affordability in clean energy incentives.  
- A **few high-end eligible models** exceed **$200,000**, but they are outliers in the dataset.  
- **Non-eligible vehicles** include extreme high-priced models (e.g., Porsche 918 at **$845,000**), indicating luxury EVs may not qualify for incentives.  
- **Unknown eligibility vehicles** are fewer in count and mostly priced within the lower-mid range.  
- The **majority of EV incentives target affordable models**, ensuring broader adoption and accessibility.  








### 15. Are **higher-priced EVs more common in certain counties or census tracts**?

In [ ]:
# Average Base MSRP by each Counnty 
avg_baseMSSRP_county = df.groupby('County')['Base MSRP'].mean().reset_index(name='AVG Base MSRP').sort_values(by='AVG Base MSRP' , ascending=False)
# Getting only those counties whose MSRP is greater than 0
avg_baseMSSRP_county = avg_baseMSSRP_county[avg_baseMSSRP_county['AVG Base MSRP'] > 0]

#Average Base MSRP for each census Tract
avg_baseMSSRP_census = df.groupby('2020 Census Tract')['Base MSRP'].mean().reset_index(name="AVG Base MSRP").sort_values(by='AVG Base MSRP' ,
                                                                                                                          ascending=False)
## getting Only those whose Bse msrp is greater than 0
avg_baseMSSRP_census = avg_baseMSSRP_census[avg_baseMSSRP_census['AVG Base MSRP'] > 0]

In [ ]:
# Getting top 10 Counties with highest average Base MSRP
top_10_counties = avg_baseMSSRP_county.head(10)

# Creating the figure
fig = go.Figure()

# Adding bar trace
fig.add_trace(go.Bar(
    x=top_10_counties['County'],
    y=top_10_counties['AVG Base MSRP'],
    text=top_10_counties['AVG Base MSRP'],
    textposition='outside',
    marker=dict(color=top_10_counties['AVG Base MSRP'], colorscale='Blues'),
    name='Average Base MSRP'
))

# Adding a line trace to highlight trends
fig.add_trace(go.Scatter(
    x=top_10_counties['County'],
    y=top_10_counties['AVG Base MSRP'],
    mode='lines+markers',
    name='Trend Line',
    marker=dict(size=8, color='red'),
    line=dict(width=2, dash='dash')
))

# Updating layout for better readability
fig.update_layout(
    title='Top 10 Counties with Highest Average Base MSRP',
    xaxis_title='County',
    yaxis_title='Average Base MSRP',
    xaxis=dict(tickangle=-45),
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='white',
    margin=dict(l=50, r=50, t=50, b=100),
    font=dict(size=12),
    hovermode='x',
    showlegend=True
)

# Show the figure
fig.show()


- **Travis, Clackamas, and Multnomah counties** have the **highest average base MSRP**, indicating a preference for premium EV models.  
- A **gradual decline in MSRP** is observed across counties, with the **lowest-priced EVs in El Paso**.  
- **Higher-priced EVs tend to cluster in wealthier counties**, where residents may afford luxury models.  
- The presence of **EV-friendly policies, incentives, and charging infrastructure** may influence higher adoption of premium EVs in certain areas.  
- **Counties with lower average MSRP** suggest a preference for **more affordable, mass-market EV models**.  

#### **Suggestions for Management:**
1. **Target Premium EV Sales in Wealthier Counties**  
   - Focus marketing efforts for **luxury EV models** in counties like **Travis, Clackamas, and Multnomah**, where high-end EVs are in demand.  
2. **Expand Affordable EV Options in Lower-Income Counties**  
   - Introduce **incentives or financing options** to boost adoption in regions with lower average MSRP, such as **El Paso**.  
3. **Enhance Charging Infrastructure in High-Priced EV Areas**  
   - Ensure that **premium EV regions** have sufficient **charging infrastructure** to support long-range electric vehicle users.  
4. **Leverage Incentives & Rebates**  
   - Work with **local governments** to provide **tax incentives** or **rebates** to encourage adoption of both high-end and budget-friendly EVs.  
5. **Monitor Market Trends & Adapt Strategy**  
   - Continuously analyze county-level **EV adoption trends** to **align product offerings** with consumer demand.  

In [ ]:
avg_baseMSSRP_census.head(10).style.background_gradient(subset='AVG Base MSRP',cmap='Blues')

### 16. How does **CAFV (Clean Alternative Fuel Vehicle) eligibility** impact EV sales trends?

In [ ]:
# Getting count of each CAFV_records
CAFV_records = df['Clean Alternative Fuel Vehicle (CAFV) Eligibility'].value_counts().reset_index()
CAFV_records.columns = ['CAFV', 'CAFV Count'] # rename the columns

# Visualization
fig = px.pie(
    CAFV_records,
    names='CAFV',
    values='CAFV Count',
    hole=0.4,
    color_discrete_sequence=px.colors.sequential.Blues_r,
    title='Distribution Of CAFV_records',
)

fig.update_traces(textinfo="percent+label", pull=[0.05] * len(CAFV_records))  # Slightly pull out slices for emphasis
fig.update_layout(width=800, height=500, showlegend=True)
fig.show()

### Impact of CAFV Eligibility on EV Sales

CAFV (Clean Alternative Fuel Vehicle) eligibility significantly influences EV sales by affecting affordability, consumer demand, and manufacturer strategies.

- **Incentives & Demand**: Eligible vehicles attract buyers through **tax credits, rebates, and HOV lane access**, increasing adoption. In contrast, **"Not eligible"** vehicles may struggle with lower demand due to higher costs.
- **Market Uncertainty**: The **"Unknown"** category creates uncertainty, potentially slowing EV adoption.
- **Manufacturer Strategy**: Automakers align EV production with eligibility criteria, leading to a shift toward **longer-range, efficient EVs**.
- **Regional Sales Trends**: Areas with **strong CAFV support** (e.g., California) experience **higher EV sales**, while regions with fewer incentives see slower growth.

🚗⚡ Higher CAFV eligibility accelerates EV adoption, while unclear or low eligibility can hinder market expansion.


### 17. Are there specific **counties where CAFV-eligible vehicles are more widely adopted**?

In [ ]:
# Getting CAFV eligible vehicles
CAFV_eligible = df[df['Clean Alternative Fuel Vehicle (CAFV) Eligibility'] == 'Eligible']

# Total count of CAFV eligible vehicles
tot_CAFV_eligible = len(CAFV_eligible)

# Grouping by County to get EV count
CAFV_eligible_count = pd.DataFrame(CAFV_eligible['County'].value_counts())
CAFV_eligible_count.columns = ['EVs Count']

# Calculating percentage
CAFV_eligible_count['Percentage of total CAFV Eligibility'] = (
    CAFV_eligible_count['EVs Count'] * 100 / tot_CAFV_eligible
).round(2)

# Filtering out rows where percentage is greater than 0
CAFV_eligible_count = CAFV_eligible_count[CAFV_eligible_count['Percentage of total CAFV Eligibility'] > 0]

# Display the filtered DataFrame
CAFV_eligible_count.head()

In [ ]:
# Create figure
fig = go.Figure()

# Add line trace
fig.add_trace(go.Scatter(
    x=CAFV_eligible_count.index,  # Use the index if 'County' is the index
    y=CAFV_eligible_count['EVs Count'],  
    mode='lines+markers',  
    name='EV Count',
    marker=dict(size=6, color='blue'),
    line=dict(width=2, color='blue')
))

# Update layout
fig.update_layout(
    title='CAFV-Eligible EV Distribution Across Counties',
    xaxis_title='County',
    yaxis_title='EVs Count',
    xaxis=dict(tickangle=-45, showgrid=False),
    yaxis=dict(showgrid=True, zeroline=True),
    hovermode='x',
    template='plotly_white',
    margin=dict(l=50, r=50, t=50, b=100)
)

# Show plot
fig.show()

### **Insights on CAFV-Eligible EV Distribution Across Counties**

#### **1. Key Regions Driving EV Adoption**
- **King County dominates EV adoption** with **48.79%** of all CAFV-eligible EVs, followed by Snohomish (10.62%) and Pierce (7.75%).  
- These counties likely have **strong incentives, better EV infrastructure, and higher consumer interest** in clean energy vehicles.  

#### **2. Growth Opportunities in Mid-Tier Counties**
- Counties such as **Clark (6.31%), Thurston (3.94%), and Kitsap (3.64%)** have moderate EV adoption rates.  
- Targeted policies, such as **increased charging stations, financial incentives, and EV awareness campaigns**, could further drive adoption.  

#### **3. Areas with Low EV Penetration**
- Several counties, including **Whitman (0.19%), Pacific (0.17%), and Okanogan (0.16%)**, have minimal EV adoption.  
- Some counties, such as **Pend Oreille (0.03%) and Ferry (0.02%)**, show extremely low uptake, highlighting **significant infrastructure or awareness gaps**.  
- Out-of-state counties like **San Diego, Fairfax, and Montgomery** have **negligible** presence, indicating potential cross-border market opportunities.  

#### **4. Strategic Recommendations**
- **Expand EV incentives**: Provide subsidies or tax benefits in mid and low-adoption counties.  
- **Improve charging infrastructure**: Address infrastructure gaps in low-EV-adoption regions.  
- **Enhance public awareness**: Promote EV benefits in underperforming regions through targeted outreach.  

By focusing on these areas, we can **maximize EV adoption, optimize infrastructure investments, and align strategies** with regional demands. 🚗⚡  


### 18. How has the **EV market share changed over time (by model year)?**

In [ ]:
# Grouping data by Model Year and counting EVs
ev_market_share_by_year = df.groupby('Model Year').size().reset_index(name='EV Count')

# Creating the line chart
fig = px.line(
    ev_market_share_by_year,
    x='Model Year',
    y='EV Count',
    title='<b>📈 EV Market Share Over Time (by Model Year)</b>',
    labels={'Model Year': 'Model Year', 'EV Count': 'Number of EVs'},
    markers=True
)

# Enhancing traces for better visibility
fig.update_traces(
    mode='lines+markers',
    marker=dict(size=8, color='blue', opacity=0.8, symbol='circle'),
    line=dict(width=3, color='blue', dash='solid')
)

# Updating layout for better readability
fig.update_layout(
    xaxis=dict(
        title='<b>Model Year</b>',
        tickangle=-45,  # Rotating for better spacing
        showgrid=True,
        gridcolor='lightgray',
        tickmode='linear',
        tickfont=dict(size=12)
    ),
    yaxis=dict(
        title='<b>Number of EVs</b>',
        showgrid=True,
        gridcolor='lightgray',
        zeroline=True
    ),
    hovermode='x unified',  # Improved hover experience
    template='plotly_white',
    width=1100,
    height=600,
    margin=dict(l=60, r=50, t=70, b=80),
    font=dict(size=14),
    title_font=dict(size=18)  # Enlarged title font
)

# Show the plot
fig.show()


### 📊 EV Market Share Trend Over Time (by Model Year)

### 🔍 Key Insights:
- EV adoption remained **negligible before 2010**.
- A **steady increase** began around **2012**, followed by **rapid growth after 2018**.
- The **highest adoption** occurred in **2023**, surpassing **60,000 EVs**.
- A decline is observed in **2024 & 2025**, likely due to incomplete data or recent market shifts.

### 📢 Recommendations:
1. **Sustain Growth Momentum:**  
   - Expand **charging infrastructure & incentives** to maintain adoption rates.  

2. **Address Recent Decline:**  
   - Investigate reasons for the **2024–2025 drop** (policy changes, supply chain issues, or economic factors).  

3. **Encourage Newer Models:**  
   - Promote the **latest EV models** through **subsidies, awareness campaigns, and financing options**.  

Strategic actions can ensure **continued EV market expansion**. 🚀⚡  


### 19. Which **models and manufacturers qualify the most for CAFV eligibility**?

In [ ]:
# Filter the dataframe for CAFV eligible vehicles
cafv_eligible_vehicles = df[df['Clean Alternative Fuel Vehicle (CAFV) Eligibility'] == 'Eligible']

# Group by Make and Model to get the count of eligible vehicles
cafv_eligible_count = cafv_eligible_vehicles.groupby(['Make', 'Model']).size().reset_index(name='Count')

# Sort the data by count in descending order to get the top models and manufacturers
cafv_eligible_count = cafv_eligible_count.sort_values(by='Count', ascending=False).head(10)

# Visualization
fig = px.bar(
    cafv_eligible_count,
    x='Model',
    y='Count',
    color='Make',
    text='Count',
    title='Top 10 CAFV Eligible Electric Vehicle Models by Count',
    labels={'Model': 'EV Model', 'Count': 'Number of Vehicles', 'Make': 'Manufacturer'},
    color_discrete_sequence=px.colors.sequential.Blues_r
)

fig.update_traces(
    textposition='outside',
    marker=dict(line=dict(width=1, color='black'))
)

fig.update_layout(
    xaxis=dict(title='EV Model', tickangle=-45),
    yaxis=dict(title='Number of Vehicles', showgrid=True),
    hovermode='x',
    template='plotly_white',
    width=1000,
    height=600,
    margin=dict(l=50, r=50, t=70, b=100),
    font=dict(size=14)
)

fig.show()

### 🚗 Top CAFV-Eligible EV Models & Manufacturers

### 🔍 Key Insights:
- **Tesla dominates** the CAFV eligibility list, with **Model 3 (13,506)** leading, followed by **Model S (5,849)** and **Model X (3,221)**.
- **Nissan Leaf** ranks second overall with **10,369** vehicles, showing strong adoption.
- **Chevrolet’s Bolt EV (4,760) and Volt (4,720)** also have significant CAFV-qualified counts.
- **Toyota, Chrysler, and BMW** have moderate representation, with **RAV4 Prime (3,515), Pacifica (3,733), and X5 (2,328)**.

### 📢 Recommendations:
1. **Leverage Tesla’s Popularity**  
   - Expand Tesla’s CAFV eligibility programs to sustain market leadership.  

2. **Encourage Nissan & Chevrolet**  
   - Promote **Nissan Leaf and Chevrolet EVs** through **tax incentives** and **subsidies**.  

3. **Increase Model Variety**  
   - Encourage **other manufacturers (Toyota, Chrysler, BMW)** to introduce **more CAFV-eligible models**.  

A diverse EV ecosystem will boost **CAFV adoption & sustainability goals**. ⚡🌱  


### **4️⃣ Environmental & Regulatory Considerations**
#### 20. What is the impact of **local policies or incentives** on EV adoption rates in different counties?

In [ ]:
# Grouping data by County to get the average Base MSRP and count of EVs
ev_adoption_incentives = df.groupby('County').agg({'Base MSRP': 'mean', 'VIN (1-10)': 'count'}).reset_index()
ev_adoption_incentives.columns = ['County', 'Avg Base MSRP', 'EV Count']

# Sorting the data by EV Count in descending order to identify top counties
ev_adoption_incentives = ev_adoption_incentives.sort_values(by='EV Count', ascending=False)

# Visualization
fig = px.scatter(
    ev_adoption_incentives,
    x='Avg Base MSRP',
    y='EV Count',
    size='EV Count',
    color='EV Count',
    hover_name='County',
    title='Impact of Local Policies or Incentives on EV Adoption Rates',
    labels={'Avg Base MSRP': 'Average Base MSRP', 'EV Count': 'EV Count'},
    template='plotly_white'
)

fig.update_traces(marker=dict(line=dict(width=1, color='black')))
fig.update_layout(
    xaxis=dict(showgrid=True, gridcolor='lightgray'),
    yaxis=dict(showgrid=True, gridcolor='lightgray'),
    width=1000,
    height=600
)

fig.show()

### Analysis of Local Policies and Incentives on EV Adoption Rates

The scatter plot visualizes the relationship between **Average Base MSRP (Manufacturer's Suggested Retail Price)** and **EV adoption rates** across different counties. The key insights are:

- **Higher EV Adoption with Incentives:** Counties with strong **local policies** or **financial incentives** show significantly **higher EV adoption rates**, even at lower price ranges.
- **Substantial Adoption in Lower MSRP Range:** The majority of EVs fall within a **lower MSRP range**, suggesting that affordability plays a crucial role in adoption.
- **Policy-Driven High EV Count:** The largest bubble represents a county with a significantly high **EV count**, likely driven by robust **incentives, subsidies, or infrastructure support**.

### Conclusion
The data suggests that **local policies, rebates, and incentives** significantly impact EV adoption, particularly in counties with strong government support. Future adoption strategies should focus on **reducing costs and expanding incentives** to encourage wider adoption.


# Final Insights and Management Recommendations
---

## **Management Recommendations:**
- **Expand Charging Infrastructure**: Prioritize counties where PHEVs dominate and BEV adoption lags.
- **Targeted Incentives**: Offer rebates in regions with lower EV adoption to boost affordability.
- **Promote Affordable EV Models**: Highlight budget-friendly options to encourage adoption.
- **Data-Driven Expansion Strategy**: Focus marketing and dealership expansion in high-income counties.
